In [43]:
import numpy as np
import sklearn
from sklearn import datasets
from sklearn.semi_supervised import LabelSpreading, SelfTrainingClassifier
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

# iris = datasets.load_iris()
# rng = np.random.RandomState(42)
# random_unlabeled_points = rng.rand(len(iris.target)) < 0.3
# labels = np.copy(iris.target)
# labels[random_unlabeled_points] = -1
# label_prop_model.fit(iris.data, labels)

In [18]:
import os
datapath =  r'D:\projects\congress\processed\training_vectors\debatepol/'


In [19]:

ds_all = np.load(os.path.join(datapath, 'sentence_vectors_guns_nomoder.npy'))
parties_all = np.load(os.path.join(datapath, 'stance_lib_binary_guns_nomoder.npy'))
stance_all = np.load(os.path.join(datapath, 'stance_lib_7scale_guns_nomoder.npy'))
thetas_hard_all = np.load(os.path.join(datapath, 'topics_hard_guns_nomoder.npy'))
As_all = np.load(os.path.join(datapath, 'author_dummies_guns_nomoder.npy')) 
theta_categ= np.argmax(thetas_hard_all, axis = -1)
ds_all = np.float32(ds_all)
thetas_all = thetas_hard_all

In [20]:
ds_all.shape

(52675, 300)

In [21]:
def sample(parties_all, theta_categ, stance_all,biased = True, notmask = 0.8, extreme = 5):

    totalsamp = len(parties_all)

    if not biased: 
        sss = StratifiedShuffleSplit(n_splits=5, test_size= 1- notmask)
        train_id, test_id = next(sss.split(theta_categ, parties_all))
    else:
        p = np.array([(4- i)**extreme  if i <4 else (i -4)**extreme for i in stance_all])
        p = p / np.sum(p)
        train_id = np.random.choice(np.arange(totalsamp), int(totalsamp * notmask), replace = False, p = p)
        
#         orderid = np.argsort(stance_all)
#         tbn = int(len(parties_all) * notmask/2)
#         train_id = np.concatenate([orderid[:tbn], orderid[-tbn:]])
        
        test_id = [i for i in range(totalsamp) if i not in train_id]

    return train_id, test_id

In [22]:
trainid, testid = sample(parties_all, theta_categ, stance_all,biased = True, notmask =0.2)

In [23]:
len(trainid) + len(testid)

52675

In [24]:
# rounds = 6
# notmasks = [0.01, 0.03, 0.05, 0.08, 0.2, 0.4, 0.6, 0.8]

# # notmasks = [0.8]
# topn = 500000
# model_name = 'label_spreading'
# label_prop_model = LabelSpreading(kernel= 'knn', n_neighbors=10)
# for biased in [True, False]:
#   bia = 'bias'
#   if not biased:
#     bia = 'unbias'
#   f = open( r'./results/SKSSL/{}_{}.txt'.format(model_name, bia), 'w')
#   for notmask in notmasks:
#     version = "{}_{}_{}".format(model_name, notmask, bia)
#     print(version)
#     if biased:
# #       continue
#       trainid, testid = sample(labels, topics, biased = biased, notmask = notmask)

#       ys = np.copy(labels)
#       ys[testid] = -1

#       Xs = np.copy(sentvec_flt)
#       Xidx = np.arange(len(Xs))
      
#       np.random.shuffle(Xidx)
#       Xs = Xs[Xidx]
#       ys = ys[Xidx]
#       print(ys[:100])
#       label_prop_model.fit(Xs[:topn], ys[:topn])
#       Xt = sentvec_flt[testid]
#       yt = labels[testid]
#       y_pred = label_prop_model.predict(Xt)
#       acc = np.mean(y_pred == yt)
#       print(acc)
#       f.write(version)
#       f.write('\t'+str(acc))
#       f.write('\n')
#     else:
#       accs = []
#       for __ in range(rounds):
#         trainid, testid = sample(labels, topics, biased = biased, notmask = notmask)
#         ys = np.copy(labels)
#         ys[testid] = -1
        
#         Xs = np.copy(sentvec_flt)
#         Xidx = np.arange(len(Xs))
      
#         np.random.shuffle(Xidx)
#         Xs = Xs[Xidx]
#         ys = ys[Xidx]
#         print(ys[:100])
#         label_prop_model.fit(Xs[:topn], ys[:topn])
#         Xt = sentvec_flt[testid]
#         yt = labels[testid]
#         y_pred = label_prop_model.predict(Xt)
#         acc = np.mean(y_pred == yt)
#         print(acc)
#         accs.append(acc)
#       f.write(version)
#       for ac in accs:
#         f.write('\t'+str(ac))
#       f.write('\n')
#   f.close()

In [53]:
def predict_party_membership(s_pred, y, As):
    # filter out nan
    
    fin = np.isfinite(s_pred)
    s_pred = s_pred[fin]
    y = y[fin]
    As = As[fin]
    nonzero_inx = (np.sum(As.T, axis = -1) !=0)
    ave_slants = (np.matmul(As.T, s_pred)/(np.sum(As.T, axis = -1) + 0.000001))[nonzero_inx]
    y_ = np.matmul(As.T, y) >0 # this is party label per person, but those with 0 speech be 0
    print("test slant", ave_slants.mean(), ave_slants[:5])
    y_true = y_[nonzero_inx]
    y_pred = ave_slants>0
    return np.mean(y_true == y_pred)

In [54]:
# self training
extreme = 5

In [55]:
# rounds = 5
# notmasks = [0.01, 0.03, 0.05, 0.08, 0.1, 0.2, 0.4, 0.6, 0.8]

# # notmasks = [0.8]
# topn = 5000000
# model_name = 'self_training'

# # svc = SVC(probability=True, gamma="auto")
# clf = RandomForestClassifier(max_depth=5, random_state=0)
# self_training_model = SelfTrainingClassifier(clf)
# # for biased in [True, False]:
# for biased in [True, False]:
#   bia = 'bias'
#   if not biased:
#     bia = 'unbias'
#   f = open( r'D:\projects\congress\results\debatepol\train\SSL/{}_{}.txt'.format(model_name, bia), 'w')
#   f.write('\taccs\tpccs\n')
#   for notmask in notmasks:
#     version = "{}_{}_{}".format(model_name, notmask, bia)
#     print(version)

#     if biased:
#       accs = []
#       pccs = []
#       for __ in range(rounds):
#         trainid, testid = sample(parties_all, theta_categ, stance_all, biased = biased, notmask = notmask, extreme = extreme)
#         ys = np.copy(parties_all)
#         ys[testid] = -1

#         Xs = np.copy(ds_all)
#         Xidx = np.arange(len(Xs))
      
#         np.random.shuffle(Xidx)
#         Xs = Xs[Xidx]
#         ys = ys[Xidx]
#         print(ys[:100])
#         self_training_model.fit(Xs[:topn], ys[:topn])
#         Xt = ds_all[testid]
#         yt = parties_all[testid]
#         At = As_all[testid]
#         y_pred = self_training_model.predict(Xt)
#         proba_pred = self_training_model.predict_proba(Xt)
#         s_pred = np.array([np.log(i[1]*1000+ 0.00000001)- np.log(i[0]*1000+0.00000001) for i in proba_pred])

#         acc = np.mean(y_pred == yt)
#         accs.append(acc)
        
#         pcc = predict_party_membership(s_pred, yt)
#         pccs.append(pcc)
        
        
#       f.write(version)

#       f.write('\t' + str(accs))
#       f.write('\t' + str(pccs))
#       f.write('\n')
#     else:
#       accs = []
#       pccs= []
#       for __ in range(rounds):
#         trainid, testid = sample(parties_all, theta_categ, stance_all, biased = biased, notmask = notmask, extreme = extreme)
#         ys = np.copy(parties_all)
#         ys[testid] = -1

#         Xs = np.copy(ds_all)
#         Xidx = np.arange(len(Xs))
      
#         np.random.shuffle(Xidx)
#         Xs = Xs[Xidx]
#         ys = ys[Xidx]
#         print(ys[:100])
#         self_training_model.fit(Xs[:topn], ys[:topn])
#         Xt = ds_all[testid]
#         yt = parties_all[testid]
#         At = As_all[testid]
#         y_pred = self_training_model.predict(Xt)
#         proba_pred = self_training_model.predict_proba(Xt)
#         s_pred = np.array([np.log(i[1]*1000+ 0.00000001)- np.log(i[0]*1000+0.00000001) for i in proba_pred])

#         acc = np.mean(y_pred == yt)
#         accs.append(acc)
        
#         pcc = predict_party_membership(s_pred, yt)
#         pccs.append(pcc)
        
        
#       f.write(version)

#       f.write('\t' + str(accs))
#       f.write('\t' + str(pccs))
#       f.write('\n')
#   f.close()

In [56]:
# label spreading
def rbf_kernel_safe(X, Y=None, gamma=None): 

      X, Y = sklearn.metrics.pairwise.check_pairwise_arrays(X, Y) 
      if gamma is None: 
          gamma = 1.0 / X.shape[1] 

      K = sklearn.metrics.pairwise.euclidean_distances(X, Y, squared=True) 
      K *= -gamma 
      K -= K.max()
      np.exp(K, K)    # exponentiate K in-place 
      return K 

In [67]:
rounds = 5
topn = 500000
notmasks = [0.01, 0.03, 0.05, 0.08, 0.1, 0.2, 0.4, 0.6, 0.8]

# notmasks = [0.01]   
# topn = 5000
model_name = 'label_spreading'
label_prop_model = LabelSpreading(kernel= 'knn', n_neighbors=15,  max_iter=200)
# label_prop_model = LabelSpreading(kernel= rbf_kernel_safe,  max_iter=200)
# for biased in [True, False]:
for biased in [True, False]:
  bia = 'bias'
  if not biased:
    bia = 'unbias'
  f = open( r'D:\projects\congress\results\debatepol\train\SSL/{}_{}_1.txt'.format(model_name, bia), 'w')
  f.write('\taccs\tpccs\n')
  for notmask in notmasks:
    version = "{}_{}_{}".format(model_name, notmask, bia)
    print(version)
    
    if biased:
      accs = []
      pccs = []
      for __ in range(rounds):
        trainid, testid = sample(parties_all, theta_categ, stance_all, biased = biased, notmask = notmask, extreme = extreme)
        ys = np.copy(parties_all)
        ys[testid] = -1

        Xs = np.copy(ds_all)
        Xidx = np.arange(len(Xs))
      
        np.random.shuffle(Xidx)
        Xs = Xs[Xidx]
        ys = ys[Xidx]
        print(ys[:100])
        label_prop_model.fit(Xs[:topn], ys[:topn])
        Xt = ds_all[testid]
        yt = parties_all[testid]
        At = As_all[testid]
        y_pred = label_prop_model.predict(Xt)
        proba_pred = label_prop_model.predict_proba(Xt)
        s_pred = np.array([np.log(i[1]*10000)- np.log(i[0]*10000) for i in proba_pred])
        print(y_pred.mean(), len(s_pred) - np.isfinite(s_pred).sum())

        acc = np.mean(y_pred == yt)
        accs.append(acc)
        
        pcc = predict_party_membership(s_pred, yt, At)
        pccs.append(pcc)
        
        
      f.write(version)
      print(accs, pccs)
      f.write('\t' + str(accs))
      f.write('\t' + str(pccs))
      f.write('\n')
    else:
      accs = []
      pccs = []
      for __ in range(rounds):
        trainid, testid = sample(parties_all, theta_categ, stance_all, biased = biased, notmask = notmask, extreme = extreme)
        ys = np.copy(parties_all)
        ys[testid] = -1

        Xs = np.copy(ds_all)
        Xidx = np.arange(len(Xs))
      
        np.random.shuffle(Xidx)
        Xs = Xs[Xidx]
        ys = ys[Xidx]
        print(ys[:100])
        label_prop_model.fit(Xs[:topn], ys[:topn])
        Xt = ds_all[testid]
        yt = parties_all[testid]
        At = As_all[testid]
        y_pred = label_prop_model.predict(Xt)
        
        proba_pred = label_prop_model.predict_proba(Xt)
        s_pred = np.array([np.log(i[1]*10000)- np.log(i[0]*10000) for i in proba_pred])
        
        acc = np.mean(y_pred == yt)
        accs.append(acc)
        
        pcc = predict_party_membership(s_pred, yt, At)
        pccs.append(pcc)
        
        
      f.write(version)
      print(accs, pccs)

      f.write('\t' + str(accs))
      f.write('\t' + str(pccs))
      f.write('\n')
  f.close()

label_spreading_0.01_bias
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
  1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1]


D:\anaconda\envs\py37\lib\site-packages\sklearn\semi_supervised\_label_propagation.py:205: RuntimeWarning: invalid value encountered in true_divide
  probabilities /= normalizer
D:\anaconda\envs\py37\lib\site-packages\sklearn\semi_supervised\_label_propagation.py:205: RuntimeWarning: invalid value encountered in true_divide
  probabilities /= normalizer
D:\anaconda\envs\py37\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: divide by zero encountered in log


0.3077336094651863 1143
test slant -1.0108139194314745 [-0.59063573  0.03346519 -6.65937994 -2.23081506 -0.60437157]
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1  0 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1]


D:\anaconda\envs\py37\lib\site-packages\sklearn\semi_supervised\_label_propagation.py:205: RuntimeWarning: invalid value encountered in true_divide
  probabilities /= normalizer
D:\anaconda\envs\py37\lib\site-packages\sklearn\semi_supervised\_label_propagation.py:205: RuntimeWarning: invalid value encountered in true_divide
  probabilities /= normalizer
D:\anaconda\envs\py37\lib\site-packages\ipykernel_launcher.py:42: RuntimeWarning: divide by zero encountered in log


0.25055130491476346 1093
test slant -1.300433342889434 [-1.37944567 -2.16914705 -1.91788955 -2.0149242  -0.8661989 ]
[-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1]


...

test slant -0.9210511880010428 [-0.67612026 -1.71561804  0.21694673 -0.82404975 -1.25849094]
[ 1  1  1  0 -1  1  0  1  0  0  0  0  0  1  1 -1  0  0  0  1  0  1  1  0
  1 -1  0  0  1 -1  0  0 -1  0  0  1  0  1  1  0  0  0 -1  0  0 -1  1  0
  1 -1  1  0 -1  1 -1 -1  1  0  0  1 -1  0  0  0  0  0  0  0  0  0  0 -1
  1  0  0 -1  0 -1  1  1  0  0  1  1  1  0  1  0  0  1  1  1  1  0  0  0
 -1  0  1  0]


D:\anaconda\envs\py37\lib\site-packages\ipykernel_launcher.py:79: RuntimeWarning: divide by zero encountered in log


test slant -1.0415141948214508 [-0.46857449 -1.30867648 -1.98264097 -1.60177361 -0.70137344]
[ 0  1  1  1 -1  1 -1 -1  0  1  1  1 -1  1  0 -1  0  0  1  0  0  1  0  0
  1  0 -1  1  1  0  1 -1  0 -1  1  1  1  1 -1  0  1  0  1 -1  1  0  1  1
  1  0 -1  0  0  1  1  0  1  1  0  1  1  0  0  0  0 -1 -1  1  0 -1  1  0
  0 -1  0  1  1  1  0 -1 -1 -1  1  0  0  0  1  0 -1  0 -1  1  0  1  1  0
  0  0  0  0]


D:\anaconda\envs\py37\lib\site-packages\ipykernel_launcher.py:79: RuntimeWarning: divide by zero encountered in log


test slant -0.9294798771932566 [-1.4744049  -2.44162515 -2.50508392 -0.83283157 -1.68097228]
[0.6865685809207404, 0.68609397247271, 0.683531086853346, 0.6841006169909825, 0.6889416231608922] [0.6489795918367347, 0.6460905349794238, 0.6639676113360324, 0.6511627906976745, 0.6719367588932806]
